## Project data

The project data comprises 78 locations in Egypt and Isreal, each encompassing several square kilometers. Here we give a quick overview of the data but further down this cell there is more detailed information on each datasource.

For each location there are:

Sentinel-2: There are 9 available bands. These bands are: 2-Blue, 3-Green, 4-Red, 5-RedEdge1, 6-RedEdge2, 7-RedEdge3, 8-NIR, 11-SWIR1, 12-SWIR2. Notice that band 8A-RedEdge is not supplied. These images are not normalised yet, but we will see how to work and visualise them later on.

Sentinel-1: There are 2 available bands which are the VV and VH bands. VV is the mode that transmits vertical waves and receives vertical waves to create the SAR image while VH is the mode that transmits vertical waves and receives horizontal waves. The data has already been despeckled and processed to contain values in  dB .

CopDEM: The Copernicus DEM has 4 channels. The actual elevation in meters is stored in the 4th channel. The first 2 channels store the direction (aspect in sin/cos) the slope is facing while the 3rd channel has the actual slope. All the four channels are in the range [0-1] with channel 4 normalised to the height of Mt. Everest.

Buildings labels: This is the ground truth used for the building density prediction task (building density serves as proxy for population density). The labels are expressed of number of squared meters of building on a given pixel. Values are between 0-100  m2  and for a resolution of 10 m  this reflect the percentage of coverage. This data is a combination of the Google Open Buildings dataset, OSM buildings and manual labeling.

## Sentinel-1: Synthetic Aperture Radar (SAR) data

SENTINEL-1 is an imaging radar mission providing continuous all-weather, day-and-night imagery at C-band at a resolution of about 10m every 12 days. Sentinel-1 is a phase-preserving dual polarisation SAR system. It can transmit a signal in either horizontal (H) or vertical (V) polarisation, and then receive in both H and V polarisations.

SAR sensors are able to detect flooding because flat surfaces reflect (acts as a specular reflector) the signal away from the sensor, decreasing the amount of returned radiation. This causes relatively dark pixels in radar data for water areas which contrast with non-water areas.

Speckle is a general phenomenon in SAR imagery caused by the interaction of the out-of-phase waves reflected from a target resulting in a salt-and-pepper pattern. Presence of speckle in SAR images degrades the interpretability of the land features in the data. Speckle removal is necessary for quantitative, analysis and there are various filters but there exists a tradeoff between speckle removal and resolution. The images we use have already been despeckled and processed to dB values.

## Sentinel-2: Multispectral data

SENTINEL-2 is a wide-swath, high-resolution, multi-spectral imaging mission, supporting Copernicus Land Monitoring studies, including the monitoring of vegetation, soil and water cover, as well as observation of inland waterways and coastal areas. Most importantly for us is the MultiSpectral Instrument (MSI), which samples 13 spectral bands: four bands at 10 metres, six bands at 20 metres and three bands at 60 metres spatial resolution. These include the Blue, Green and Red bands which correspond to how humans see the world but also Near Infrared (NIR) and Short-wave Infrared (SWIR) which can be used to monitor vegetation, geological features and much more.

Sentinel-2 images are very valuable for population density estimations and the mapping of vulnerable areas. The infrared bands are effective to differentiate between different types of land use while the Red band is used for mapping man-made structures. The high resolution RGB images can be used to detect schools, hospitals and other vulnerable areas.

## Copernicus DEM: Digital Elevation Model data

The Copernicus DEM is a Digital Surface Model (DSM) that represents the surface of the Earth including buildings, infrastructure and vegetation. The Copernicus DEM provides digital elevation maps for Egypt at a resolution of 30m. Data to create the DEM were acquired through the TanDEM-X mission.

As one can image, DEM’s are often used for flood prediction. DEMs can be used to create flood inundation maps, which show how water will spread across the landscape during a flood event. By combining a DEM with hydrological modeling, it is possible to simulate flood events and create maps that show which areas will be affected by flooding and to what extent. DEMs can also be used to identify natural features that can affect flooding, such as ridges, valleys, and drainage basins.


## Libraries

In [1]:
# Standard library imports
import os
import sys
from pathlib import Path

# Local imports
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

## Constants

In [2]:
download_url = "https://drive.google.com/uc?id=1N-vOm_rBPXM4CbXTdHhKfjYSeJd93sL0"
output_folder = "../data/CCAI_FLOODS_DATA"

# Sentinel-1 and Sentinel-2 data
s1_data = 'https://drive.google.com/uc?id=1LrakuW_RVA8KVsls4NyesnG4pFmxu7eS'
s2_data = 'https://drive.google.com/uc?id=1lrvPBJXHYmA6Xcd9bklNRHjcP9e_zQvQ'
s_output = "../data"

## Download Data

In [6]:
# Download dataset
!gdown $download_url

Downloading...
From (original): https://drive.google.com/uc?id=1N-vOm_rBPXM4CbXTdHhKfjYSeJd93sL0
From (redirected): https://drive.google.com/uc?id=1N-vOm_rBPXM4CbXTdHhKfjYSeJd93sL0&confirm=t&uuid=358eba93-cc7e-45b0-92d0-1867de36b436
To: c:\Users\dango\OneDrive - UWE Bristol\projects\flood-monitoring\notebooks\patches_middle-east.zip

  0%|          | 0.00/395M [00:00<?, ?B/s]
  0%|          | 1.05M/395M [00:00<00:38, 10.2MB/s]
  1%|          | 3.15M/395M [00:00<00:24, 16.3MB/s]
  1%|▏         | 5.77M/395M [00:00<00:19, 20.0MB/s]
  2%|▏         | 8.91M/395M [00:00<00:16, 23.1MB/s]
  3%|▎         | 11.5M/395M [00:00<00:16, 24.0MB/s]
  4%|▎         | 14.2M/395M [00:00<00:15, 24.1MB/s]
  4%|▍         | 17.3M/395M [00:00<00:14, 25.8MB/s]
  5%|▌         | 20.4M/395M [00:00<00:13, 26.8MB/s]
  6%|▌         | 23.6M/395M [00:00<00:14, 26.3MB/s]
  7%|▋         | 26.7M/395M [00:01<00:14, 26.2MB/s]
  8%|▊         | 29.9M/395M [00:01<00:15, 22.9MB/s]
  8%|▊         | 32.5M/395M [00:01<00:16, 22.5MB/

## Unzip Data

In [7]:
# Ensure the output directory exists
Path(output_folder).mkdir(parents=True, exist_ok=True)

# Unzip the downloaded file
!unzip -oq patches_middle-east.zip -d $output_folder

# Remove the zip file
!rm patches_middle-east.zip


## Download Sentinel-1 and Sentinel-2 Data

In [8]:
# Download Sentinel-1 and Sentinel-2 data
!gdown $s1_data -O $s_output
!gdown $s2_data -O $s_output

Downloading...
From (original): https://drive.google.com/uc?id=1LrakuW_RVA8KVsls4NyesnG4pFmxu7eS
From (redirected): https://drive.google.com/uc?id=1LrakuW_RVA8KVsls4NyesnG4pFmxu7eS&confirm=t&uuid=82f01603-d2f1-41c1-bb48-91724bc86f2c
To: c:\Users\dango\OneDrive - UWE Bristol\projects\flood-monitoring\data

  0%|          | 0.00/819M [00:00<?, ?B/s]
  0%|          | 1.57M/819M [00:00<01:02, 13.0MB/s]
  1%|          | 4.72M/819M [00:00<00:36, 22.1MB/s]
  1%|          | 7.86M/819M [00:00<00:32, 24.8MB/s]
  1%|▏         | 11.0M/819M [00:00<00:30, 26.2MB/s]
  2%|▏         | 14.2M/819M [00:00<00:29, 27.1MB/s]
  2%|▏         | 17.3M/819M [00:00<00:29, 27.6MB/s]
  2%|▏         | 20.4M/819M [00:00<00:28, 28.1MB/s]
  3%|▎         | 23.6M/819M [00:00<00:28, 28.4MB/s]
  3%|▎         | 26.7M/819M [00:00<00:27, 28.6MB/s]
  4%|▎         | 29.9M/819M [00:01<00:27, 28.9MB/s]
  4%|▍         | 33.0M/819M [00:01<00:27, 29.0MB/s]
  4%|▍         | 36.2M/819M [00:01<00:26, 29.0MB/s]
  5%|▍         | 39.3M/819